In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/order.csv


*****Shopee Code League 2020 - Order Brushing (Challenge 1)*************

In [3]:
# import data
data = pd.read_csv('/kaggle/input/order.csv')
data

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [6]:
# filter and select only shop id(s) with more than 2 orders
order_freq_per_shop = data.shopid.value_counts()
order_freq_per_shop_2 = order_freq_per_shop[order_freq_per_shop > 2]
order_data = data[data['shopid'].isin(order_freq_per_shop_2.index)]

order_data

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [11]:
# convert event_time columnt to pd.datetime object. Then sort data based on shop id and event time for easy manipulation later on
order_data_2 = order_data
order_data_2['event_time'] = pd.to_datetime(order_data_2['event_time'])
order_data_2 = order_data_2.sort_values(['shopid', 'event_time']).reset_index(drop=True)

order_data_2

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,orderid,shopid,userid,event_time
0,31195675919209,10061,168750452,2019-12-28 09:27:55
1,31205132327893,10061,194819216,2019-12-28 12:05:32
2,31412115824794,10061,130633421,2019-12-30 21:35:15
3,31431527100615,10061,62464559,2019-12-31 02:58:48
4,31075686185309,10084,4401933,2019-12-27 00:08:06
...,...,...,...,...
210352,31407699385160,213900783,215251024,2019-12-30 20:21:39
210353,31468414075366,214432425,1134243,2019-12-31 13:13:34
210354,31505523761333,214432425,1134243,2019-12-31 23:32:03
210355,31506349293329,214432425,52594422,2019-12-31 23:45:49


In [12]:
# create dictionary with shop id as key and dataframe of order of that shop id as the value
list_shop = order_data_2.shopid.value_counts().index

dict_shop = {}
for shop in list_shop:
    dict_shop[shop] = order_data_2[order_data_2['shopid'] == shop]

In [14]:
# go thru each shop id to find the frauds by counting concentrate
frauds = {}
for shop in list_shop:
#     counter_shop += 1
    iter_df = dict_shop[shop]
    for time in iter_df['event_time']:
#         counter_time += 1
        test_time = time

        order_count = 0
        user_count = 0
        concentrate = 0
        
        check_df = iter_df[(iter_df['event_time'] < test_time + pd.Timedelta(hours=1)) & (iter_df['event_time'] >= test_time)]
        
        order_count = len(check_df.index)
        user_id = check_df.userid.value_counts().index
        user_count = len(user_id)
        
        if not check_df.empty:
            concentrate = order_count/user_count
        
        if concentrate >= 3:
            iter_df = iter_df.drop(iter_df[iter_df['orderid'].isin(check_df['orderid'])].index)

            if shop in frauds:
                frauds[shop].append(check_df)
            else:
                frauds[shop] = [check_df] 
    

In [16]:
# go thru the dictionary frauds to identify the user id who contributed the most in order brushing

for shopid in frauds:
#     print(shopid)
    count_user = pd.Series([])
    for item in frauds[shopid]:
        to_append = item.userid.value_counts()
#         print(type(to_append))
        if count_user.empty:
#             print("empty")
            count_user = to_append
        else:
#             print("not empty")
            count_user = pd.concat([count_user, to_append])

    count_user = count_user.reset_index().rename(columns={'index':'userid','userid':'count'})

    count_user['count'] = count_user['count']/count_user['count'].sum()

    count_user.sort_values(by=['count'], ascending=False)
    highest = count_user['count'].max()

    highest_contributor = count_user[count_user['count'] == highest]['userid']
    frauds[shopid] = highest_contributor.tolist()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


In [19]:
# at this stage, frauds contain a dictionary with shopid as key and list of userid(s) who contributed the most to order brushing as value
frauds

#convert into csv or any output desired

{43412276: [50672876],
 126639670: [101296643],
 156883302: [188025647, 76102350],
 8715449: [9753706],
 11896733: [156614746],
 76668593: [215224309],
 66337054: [122507717],
 104590058: [81928284],
 66861410: [213502289],
 96757706: [61348019],
 75622223: [69523106],
 4149345: [212325226],
 51134277: [29857724, 212200633],
 110871170: [133542654],
 12078788: [214588488],
 18190345: [89014205],
 27476241: [174145893],
 1175477: [122277324],
 27015534: [188025647],
 51487211: [214808165],
 68862371: [67554410],
 9676950: [210920501],
 143281052: [186080843, 99517130],
 22800308: [99997899],
 87621695: [87846708],
 132834615: [159124456],
 189308408: [27456547],
 96757644: [215243653],
 27121667: [183926374],
 148917764: [189631941],
 27987240: [192785138],
 119362482: [92275977],
 84866704: [65814291],
 2856708: [123959597],
 115506347: [200925208],
 29979455: [107641182],
 86822226: [211907762],
 50713918: [172106152],
 145777302: [201343856],
 136564914: [178491887],
 93358941: [7941